In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
# --- 1. Formulace vlastní úlohy a generování dat ---
# Alespoň 12 jednotek, minimálně 2 vstupy a 2 výstupy

# Počet DMU (poboček)
n_dmus = 12
dmu_names = [f"Pobočka {chr(65+i)}" for i in range(n_dmus)]

# Generování náhodných, ale smysluplných dat
np.random.seed(42) # Pro reprodukovatelnost

data = {
    'Pobočka': dmu_names,
    'Počet zaměstnanců (Vstup 1)': np.random.randint(3, 11, size=n_dmus), # Např. 3-10 zaměstnanců
    'Provozní náklady (tis. Kč) (Vstup 2)': np.random.randint(150, 501, size=n_dmus), # Např. 150-500 tis. Kč
    'Počet nových účtů (Výstup 1)': np.random.randint(20, 101, size=n_dmus), # Např. 20-100 nových účtů
    'Objem vkladů (mil. Kč) (Výstup 2)': np.round(np.random.uniform(5, 25, size=n_dmus), 1) # Např. 5-25 mil. Kč vkladů
}
df = pd.DataFrame(data)

# Úmyslně uděláme některé pobočky méně efektivní pro demonstraci
# Pobočka C: vysoké náklady, málo zaměstnanců, nízké výstupy
df.loc[df['Pobočka'] == 'Pobočka C', 'Počet zaměstnanců (Vstup 1)'] = 4
df.loc[df['Pobočka'] == 'Pobočka C', 'Provozní náklady (tis. Kč) (Vstup 2)'] = 480
df.loc[df['Pobočka'] == 'Pobočka C', 'Počet nových účtů (Výstup 1)'] = 25
df.loc[df['Pobočka'] == 'Pobočka C', 'Objem vkladů (mil. Kč) (Výstup 2)'] = 6.0

# Pobočka G: průměrné vstupy, velmi nízké výstupy
df.loc[df['Pobočka'] == 'Pobočka G', 'Počet zaměstnanců (Vstup 1)'] = 6
df.loc[df['Pobočka'] == 'Pobočka G', 'Provozní náklady (tis. Kč) (Vstup 2)'] = 300
df.loc[df['Pobočka'] == 'Pobočka G', 'Počet nových účtů (Výstup 1)'] = 30
df.loc[df['Pobočka'] == 'Pobočka G', 'Objem vkladů (mil. Kč) (Výstup 2)'] = 7.5

print("--- Data pro analýzu ---")
print(df)
print("\n")

# Definice vstupů a výstupů
inputs = ['Počet zaměstnanců (Vstup 1)', 'Provozní náklady (tis. Kč) (Vstup 2)']
outputs = ['Počet nových účtů (Výstup 1)', 'Objem vkladů (mil. Kč) (Výstup 2)']

--- Data pro analýzu ---
      Pobočka  Počet zaměstnanců (Vstup 1)  \
0   Pobočka A                            9   
1   Pobočka B                            6   
2   Pobočka C                            4   
3   Pobočka D                            9   
4   Pobočka E                            5   
5   Pobočka F                           10   
6   Pobočka G                            6   
7   Pobočka H                            7   
8   Pobočka I                            9   
9   Pobočka J                            4   
10  Pobočka K                            5   
11  Pobočka L                            9   

    Provozní náklady (tis. Kč) (Vstup 2)  Počet nových účtů (Výstup 1)  \
0                                    480                            52   
1                                    237                            95   
2                                    480                            25   
3                                    301                            41   
4     

In [3]:
# --- 2. Výpočet míry efektivnosti (CCR model, input-oriented) ---
efficiency_scores = {}
peer_weights = {} # Pro uložení vah lambda pro každou DMU_o

# Převedení dat na numpy pole pro snazší přístup
input_data = df[inputs].values
output_data = df[outputs].values
num_inputs = len(inputs)
num_outputs = len(outputs)

# Pro každou DMU (označenou jako DMU_o) řešíme LP problém
for o in range(n_dmus): # o je index DMU, kterou hodnotíme
    # Vytvoření LP problému
    prob = pulp.LpProblem(f"DEA_DMU_{o}", pulp.LpMinimize)

    # Rozhodovací proměnné
    # Theta - míra efektivity pro DMU_o
    theta = pulp.LpVariable(f"theta_{o}", lowBound=0) # Může být i highBound=1, ale pro CCR to není nutné explicitně

    # Lambdas - váhy pro ostatní DMU
    lambdas = [pulp.LpVariable(f"lambda_{o}_{j}", lowBound=0) for j in range(n_dmus)]

    # Účelová funkce: minimalizovat theta
    prob += theta, "Míra efektivity"

    # Omezující podmínky
    # Součet vážených vstupů <= theta * vstup_o
    for i in range(num_inputs):
        prob += pulp.lpSum(lambdas[j] * input_data[j, i] for j in range(n_dmus)) <= theta * input_data[o, i], f"Vstup_constraint_{i}_DMU_{o}"

    # Součet vážených výstupů >= výstup_o
    for r in range(num_outputs):
        prob += pulp.lpSum(lambdas[j] * output_data[j, r] for j in range(n_dmus)) >= output_data[o, r], f"Výstup_constraint_{r}_DMU_{o}"
    
    # Řešení problému (bez výpisů z řešiče)
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    # Uložení výsledků
    efficiency_scores[df['Pobočka'].iloc[o]] = pulp.value(theta)
    peer_weights[df['Pobočka'].iloc[o]] = {df['Pobočka'].iloc[j]: pulp.value(lambdas[j]) for j in range(n_dmus) if pulp.value(lambdas[j]) > 1e-5} # Ukládáme pouze nenulové váhy

# Přidání skórů efektivity do DataFrame
df['Efektivita (CCR)'] = df['Pobočka'].map(efficiency_scores)

print("--- Výsledky efektivity ---")
print(df[['Pobočka', 'Efektivita (CCR)']].sort_values(by='Efektivita (CCR)', ascending=False))
print("\n")

--- Výsledky efektivity ---
      Pobočka  Efektivita (CCR)
1   Pobočka B          1.000000
10  Pobočka K          1.000000
9   Pobočka J          1.000000
5   Pobočka F          0.987798
3   Pobočka D          0.897762
4   Pobočka E          0.847662
8   Pobočka I          0.805576
7   Pobočka H          0.702219
0   Pobočka A          0.693662
11  Pobočka L          0.564851
6   Pobočka G          0.408942
2   Pobočka C          0.385802




In [4]:
# --- 3. Cílové hodnoty pro neefektivní jednotky a komentář ---
print("--- Analýza neefektivních jednotek a cílové hodnoty ---")
inefficient_dmus = df[df['Efektivita (CCR)'] < 0.99999] # Používáme toleranci pro případné numerické nepřesnosti

if inefficient_dmus.empty:
    print("Všechny jednotky jsou efektivní.")
else:
    for index, row in inefficient_dmus.iterrows():
        dmu_name = row['Pobočka']
        current_efficiency = row['Efektivita (CCR)']
        print(f"\n--- Pobočka: {dmu_name} (Efektivita: {current_efficiency:.4f}) ---")

        print("Cílové hodnoty vstupů (pro dosažení efektivity 1.0 při stávajících výstupech):")
        for i_idx, input_name in enumerate(inputs):
            current_input_value = input_data[index, i_idx]
            target_input_value = current_input_value * current_efficiency
            reduction_needed = current_input_value - target_input_value
            print(f"  {input_name}:")
            print(f"    Současná hodnota: {current_input_value:.2f}")
            print(f"    Cílová hodnota:   {target_input_value:.2f} (snížení o {reduction_needed:.2f}, tj. o {(1-current_efficiency)*100:.2f}%)")

        print("Referenční jednotky (peers) a jejich váhy (lambda):")
        if dmu_name in peer_weights and peer_weights[dmu_name]:
            for peer, weight in peer_weights[dmu_name].items():
                print(f"  - {peer}: {weight:.4f}")
            
            # Výpočet vážených vstupů a výstupů referenční jednotky (virtuální efektivní jednotky)
            # Toto je jednotka na hranici efektivity, kterou by neefektivní jednotka měla napodobit
            print("Charakteristiky virtuální efektivní jednotky (složené z peers):")
            for i_idx, input_name in enumerate(inputs):
                weighted_input = sum(peer_weights[dmu_name].get(df['Pobočka'].iloc[j], 0) * input_data[j, i_idx] for j in range(n_dmus))
                print(f"  {input_name} (vážený průměr peers): {weighted_input:.2f}")
            
            for o_idx, output_name in enumerate(outputs):
                weighted_output = sum(peer_weights[dmu_name].get(df['Pobočka'].iloc[j], 0) * output_data[j, o_idx] for j in range(n_dmus))
                print(f"  {output_name} (vážený průměr peers): {weighted_output:.2f} (mělo by být >= {output_data[index, o_idx]:.2f})")
        else:
            print("  Nebyly nalezeny žádné specifické referenční jednotky s nenulovou váhou (může se stát u některých konfigurací dat).")

--- Analýza neefektivních jednotek a cílové hodnoty ---

--- Pobočka: Pobočka A (Efektivita: 0.6937) ---
Cílové hodnoty vstupů (pro dosažení efektivity 1.0 při stávajících výstupech):
  Počet zaměstnanců (Vstup 1):
    Současná hodnota: 9.00
    Cílová hodnota:   6.24 (snížení o 2.76, tj. o 30.63%)
  Provozní náklady (tis. Kč) (Vstup 2):
    Současná hodnota: 480.00
    Cílová hodnota:   332.96 (snížení o 147.04, tj. o 30.63%)
Referenční jednotky (peers) a jejich váhy (lambda):
  - Pobočka K: 0.9764
Charakteristiky virtuální efektivní jednotky (složené z peers):
  Počet zaměstnanců (Vstup 1) (vážený průměr peers): 4.88
  Provozní náklady (tis. Kč) (Vstup 2) (vážený průměr peers): 332.96
  Počet nových účtů (Výstup 1) (vážený průměr peers): 79.09 (mělo by být >= 52.00)
  Objem vkladů (mil. Kč) (Výstup 2) (vážený průměr peers): 20.70 (mělo by být >= 20.70)

--- Pobočka: Pobočka C (Efektivita: 0.3858) ---
Cílové hodnoty vstupů (pro dosažení efektivity 1.0 při stávajících výstupech):
  Poč

In [5]:
print("\n\n--- Komentář k získaným výsledkům ---")
# Celkový komentář
# Kolik jednotek je efektivních?
num_efficient = df[df['Efektivita (CCR)'] >= 0.99999].shape[0]
print(f"Celkem bylo hodnoceno {n_dmus} poboček.")
print(f"{num_efficient} poboček je 100% efektivních podle CCR modelu (input-oriented).")
print(f"{n_dmus - num_efficient} poboček je neefektivních a mají potenciál pro zlepšení.")

# print("\nPobočky s nejvyšší efektivitou jsou benchmarkem pro ostatní.")
# print("Neefektivní pobočky mohou dosáhnout plné efektivity snížením svých vstupů na cílové hodnoty, aniž by snížily své výstupy.")
# print("Například, pokud má pobočka efektivitu 0.80, znamená to, že by teoreticky mohla dosáhnout stejných výstupů s 80% svých současných vstupů, tedy snížit vstupy o 20%.")
# print("Analýza 'peerů' ukazuje, ke kterým efektivním pobočkám (nebo jejich kombinaci) by se měla neefektivní pobočka přiblížit svými provozními charakteristikami.")
# print("Například Pobočka C a Pobočka G vykazují nízkou efektivitu kvůli relativně vysokým vstupům v poměru k jejich výstupům, nebo jednoduše nízkým výstupům při daných vstupech.")
# print("Je důležité poznamenat, že DEA je citlivá na výběr vstupů a výstupů a na kvalitu dat. Výsledky by měly být interpretovány v kontextu a případně doplněny další kvalitativní analýzou.")
# print("Tento CCR model předpokládá konstantní výnosy z rozsahu (Constant Returns to Scale - CRS). Pro analýzu, která zohledňuje variabilní výnosy z rozsahu (Variable Returns to Scale - VRS), by bylo nutné použít BCC model.")



--- Komentář k získaným výsledkům ---
Celkem bylo hodnoceno 12 poboček.
3 poboček je 100% efektivních podle CCR modelu (input-oriented).
9 poboček je neefektivních a mají potenciál pro zlepšení.
